In [ ]:
%pip install optuna kneed

In [ ]:
import gc, json, pickle, warnings, optuna, joblib
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from kneed import KneeLocator
from lightgbm import LGBMRegressor
from google.colab import drive
from joblib import Parallel, delayed
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import silhouette_score
from sklearn.model_selection import TimeSeriesSplit
from collections import defaultdict
from scipy.stats import pearsonr, zscore
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

In [ ]:
drive.mount('/content/drive')
warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn.utils.deprecation")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class DataLoader:
  def __init__(self, data_path, file_name):
    self.data_path = data_path
    self.file_name = file_name

  def load(self):
    df = pd.read_parquet(self.data_path + self.file_name)
    self.df = df.loc[:, ~((df == -np.inf).any() | (df == 0).all())]
    print('Data loaded')
    return self

  def train_split(self):
    label = self.df['label']
    qty_vol = self.df[['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume']]
    X_ = self.df.drop(['label', 'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume'], axis=1)
    return X_, qty_vol, label

  def test_split(self):
    qty_vol = self.df[['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume']]
    X_ = self.df.drop(['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume'], axis=1)
    return X_, qty_vol

  def load_features_json_list(self):
    with open(self.data_path + self.file_name, 'r') as f:
      return json.load(f)

  def load_clusters_pkl(self):
    with open(self.data_path + self.file_name, 'rb') as f:
      return pickle.load(f)

In [ ]:
class FeatureGenerator:
  def __init__(self, df, epsilon=1e-6):
    self.df = df
    self.epsilon = epsilon

  def develop_features_from_qty_volume(self):
    '''
      Function to develop features from bid_qty, ask_qty, buy_qty, sell_qty
      and volume columns.
    '''
    imbalance = (self.df['bid_qty'] - self.df['ask_qty']) / (self.df['bid_qty'] + self.df['ask_qty'] + self.epsilon)
    buy_sell_ratio = np.log1p(self.df['buy_qty'] / (self.df['sell_qty'] + self.epsilon))
    volume_z = zscore(self.df['volume'])

    return pd.DataFrame({
        'imbalance': imbalance,
        'buy_sell_ratio': buy_sell_ratio,
        'volume_z': volume_z
        }, index=self.df.index)

  def standardize_columns(self, columns):
    '''Function to standardize specific columns wihtin a df.'''
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(self.df[columns])
    df_scaled = pd.DataFrame(df_scaled, columns=columns, index=self.df.index)
    return pd.concat([df_scaled, self.df['label']], axis=1)

  def standardize_df(self):
    '''Function to standardize a whole df.'''
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(self.df)
    return pd.DataFrame(df_scaled, columns=self.df.columns, index=self.df.index)

In [ ]:
class LagCreator:
  def __init__(self, lag_periods = 5):
    self.lag_periods = lag_periods

  def lag_features(self, X):

    lagged_columns = []
    for col in X.columns:
        for lag in range(1, self.lag_periods + 1):
          shifted = X[col].shift(lag)
          shifted.name = f"{col}_lag{lag}"
          lagged_columns.append(shifted)

    lagged = pd.concat(lagged_columns, axis=1)
    lagged = pd.concat([X, lagged], axis=1)
    return lagged.dropna()

  def match_index_lags(self, X_lagged, Y):
    '''Y is series with a single column'''
    y_lagged = Y.loc[X_lagged.index]

    return y_lagged


In [ ]:
class FeatureLagSelector:
    def __init__(self, feature_list):
        '''
        Initialize the selector with a list of desired features.
        Features can be raw (e.g., 'X264') or lagged (e.g., 'X264_lag2')
        '''
        self.feature_list = feature_list
        self.selected_features = {}

    def _parse_feature(self, feature_name):

        '''Parse feature name into base column and lag value (if present).'''

        if '_lag' in feature_name:
            base, lag = feature_name.split('_lag')
            return base, int(lag)
        return feature_name, None

    def _transform(self, df):
        '''Build a new DataFrame with the selected and lagged features.'''

        self.selected_features.clear()
        for feature in self.feature_list:
            base, lag = self._parse_feature(feature)
            if base not in df.columns:
                print(f"Base column '{base}' not found in DataFrame")
                continue
            if lag is None:
                self.selected_features[feature] = df[base]
            else:
                self.selected_features[feature] = df[base].shift(lag)

        return pd.DataFrame(self.selected_features)

    def fit_transform(self, df):
        return self._transform(df)


In [ ]:
class ModelPipelineOptimizerLGB:
  def __init__ (self, y, df, features2, clusters_components, model_save_location, model_name):
    self.df = df
    self.features2 = features2
    self.y = y
    self.clusters_components = clusters_components
    self.data_path = model_save_location
    self.model_name = model_name

    self.best_features = [f for group in self.clusters_components[0] for f in group]
    self.X = FeatureLagSelector(self.best_features).fit_transform(self.df).dropna()
    self.features2 = LagCreator(3).lag_features(self.features2)

    self.X = pd.concat([self.X, self.features2], axis=1)

    self.y = self.y.loc[self.X.index]


    del self.df, self.features2
    gc.collect()

  def create_column_transformer(self):
    feature_clusters, pca_components = self.clusters_components
    transformers = []

    for i, (features, n_comp) in enumerate(zip(feature_clusters, pca_components)):
      if len(features) == 1:
            # For single feature, just apply StandardScaler
            pipe = Pipeline([
                ('scaler', StandardScaler())
            ])
      else:
            # Multiple features: apply scaler + PCA
            pipe = Pipeline([
                ('scaler', StandardScaler()),
                ('pca', PCA(n_components=n_comp))
            ])
      transformers.append((f'cluster_{i}', pipe, features))

    return ColumnTransformer(transformers, remainder='passthrough')

  def objective(self, trial):
    lgb_params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "objective": "regression",
        "verbosity": -1
        }


    col_transformer = self.create_column_transformer()
    tscv = TimeSeriesSplit(n_splits=10)
    scores = []

    for train_idx, val_idx in tscv.split(self.X):
      X_train, X_val = self.X.iloc[train_idx], self.X.iloc[val_idx]
      y_train, y_val = self.y.iloc[train_idx], self.y.iloc[val_idx]

      pipeline = Pipeline([
          ('pca_clusters', col_transformer),
          ('lgb', LGBMRegressor(**lgb_params))
      ])

      pipeline.fit(X_train, y_train)
      preds = pipeline.predict(X_val)

      corr, _ = pearsonr(y_val, preds)
      scores.append(corr)

    return np.mean(scores)

  def optimize(self, n_trials=50):
    self.study = optuna.create_study(direction="maximize")
    self.study.optimize(self.objective, n_trials=n_trials, n_jobs=-1)

    print("\nBest trial:")
    print(f"  Pearson correlation: {self.study.best_value:.4f}")
    print("  Hyperparameters:")
    for key, val in self.study.best_params.items():
        print(f"    {key}: {val}")

    # Visualize optimization
    optuna.visualization.plot_optimization_history(self.study).show()
    optuna.visualization.plot_param_importances(self.study).show()

    # Recreate PCA + scaler transformer
    col_transformer = self.create_column_transformer()

    # Build final pipeline with best LGB params
    best_params = self.study.best_params.copy()
    best_params.update({
        "random_state": 42,
        "n_jobs": -1,
        "objective": "regression",
        "verbosity": -1
        })

    final_pipeline = Pipeline([
        ('pca_clusters', col_transformer),
        ('lgb', LGBMRegressor(**best_params))
    ])

    # Fit final pipeline on all data
    final_pipeline.fit(self.X, self.y)

    # Save pipeline
    joblib.dump(final_pipeline, self.data_path + self.model_name)
    print(f"\nModel saved to: {self.data_path + self.model_name}")

    return final_pipeline

In [ ]:
# Features
# Group 1 - X1, X2 ....
# Group 2 - _qty, vol

In [ ]:
# Filepaths
dataPathLoad = '/content/drive/MyDrive/Colab Notebooks/DRW/data/'
dataPathSave = '/content/drive/MyDrive/Colab Notebooks/DRW/data/v7'
fileNameTrain = 'train.parquet'

In [ ]:
# Load train data and remove -inf columns and columns full of 0
loader = DataLoader(dataPathLoad, fileNameTrain)
features_group1, features_group2, label = loader.load().train_split()
del loader
gc.collect()

Data loaded


13

In [ ]:
# Build the right features2
features_group2 = FeatureGenerator(features_group2).develop_features_from_qty_volume()


In [ ]:
# Load the groups
with open(dataPathSave + 'PCA_components7.json', 'r') as f:
    groups_components = json.load(f)

In [ ]:
# Optimize and save model
optimizer = ModelPipelineOptimizerLGB(label, features_group1, features_group2, groups_components, dataPathSave, 'model7_10splits_LGB.pkl')
final_pipeline = optimizer.optimize()

[I 2025-06-07 14:59:02,306] A new study created in memory with name: no-name-9c969c04-689e-474d-974b-4fdf0389161d
[I 2025-06-07 15:22:55,682] Trial 3 finished with value: 0.00982035573026647 and parameters: {'n_estimators': 81, 'max_depth': 3, 'learning_rate': 0.004047000888822234, 'subsample': 0.9377602343470091, 'colsample_bytree': 0.6620154271155032, 'num_leaves': 78, 'min_child_samples': 16, 'reg_alpha': 2.2979574028050873, 'reg_lambda': 0.4345119249561048}. Best is trial 3 with value: 0.00982035573026647.
[I 2025-06-07 15:23:19,025] Trial 5 finished with value: 0.030173618092506016 and parameters: {'n_estimators': 196, 'max_depth': 4, 'learning_rate': 0.007014923996247437, 'subsample': 0.9458060317127609, 'colsample_bytree': 0.5438642663394593, 'num_leaves': 115, 'min_child_samples': 87, 'reg_alpha': 0.003957447778671953, 'reg_lambda': 4.22884782638347}. Best is trial 5 with value: 0.030173618092506016.
[I 2025-06-07 15:24:59,670] Trial 0 finished with value: 0.044411801016120425 